## BeautifulSoup

In [1]:
import requests
from bs4 import BeautifulSoup

In [20]:
bad = 'ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА, И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА'

In [55]:
def make_clean_text_with_url(message):
    soup = BeautifulSoup(message)
    
    first_a = soup.find('a')
    try:
        first_link = first_a.get('href')
    except AttributeError:
        first_link = 'NaN'
    
    text = soup.get_text()
    text = text.replace("\xa0", '').replace("\n", '')
    text = text.replace(bad, '\n')
    
    return {'raw_news': text, 'first_link': first_link}

In [60]:
# Нужен контроль за тем, чтобы не парсинг не дублировался.

def make_articles_chunk_dict(tlg_channel_name):
    answer = requests.get('https://tg.i-c-a.su/json/' + tlg_channel_name)
    data = answer.json()
    draft_articles = [make_clean_text_with_url(data['messages'][el]['message']) for el in range(len(data['messages']))]
    id_articles = [data['messages'][el]['id'] for el in range(len(data['messages']))]
    draft_articles_dict = {id_articles[el]: draft_articles[el] for el in range(len(draft_articles))}
    return draft_articles_dict

In [57]:
raw_articles_dict = make_articles_chunk_dict('meduzalive')

In [53]:
len(raw_articles_dict)

10

In [67]:
for v in raw_articles_dict.values():
    print(len(v['raw_news']))

548
880
518
441
676
954
360
231
740
660


## Trafilatura

In [69]:
!pip install trafilatura

  Attempting uninstall: lxml
    Found existing installation: lxml 4.6.1
    Uninstalling lxml-4.6.1:
      Successfully uninstalled lxml-4.6.1


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Отказано в доступе: 'C:\\Users\\maxle\\anaconda3\\Lib\\site-packages\\~xml\\etree.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [82]:
!pip install -U trafilatura

Requirement already up-to-date: trafilatura in c:\users\maxle\appdata\roaming\python\python38\site-packages (1.2.2)


In [70]:
!pip install trafilatura --user

  Using cached trafilatura-1.2.2-py3-none-any.whl (1.0 MB)
  Using cached charset_normalizer-2.1.0-py3-none-any.whl (39 kB)
  Using cached courlan-0.7.2-py3-none-any.whl (32 kB)
  Using cached htmldate-1.2.3-py3-none-any.whl (38 kB)
  Using cached jusText-3.0.0-py2.py3-none-any.whl (837 kB)
  Using cached tld-0.12.6-py38-none-any.whl (412 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached dateparser-1.1.1-py2.py3-none-any.whl (288 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
  Using cached pytz_deprecation_shim-0.1.0.post0-py2.py3-none-any.whl (15 kB)
  Using cached backports.zoneinfo-0.2.1-cp38-cp38-win_amd64.whl (38 kB)
  Using cached tzdata-2022.1-py2.py3-none-any.whl (339 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

conda 4.1

In [102]:
import trafilatura
from trafilatura import feeds
from trafilatura import sitemaps
from trafilatura import bare_extraction

In [137]:
downloaded = trafilatura.fetch_url('https://tg.i-c-a.su/json/meduzalive')

In [152]:
result = trafilatura.extract(downloaded, output_format='json', include_links=True)

str

In [157]:
import json

In [171]:
wow = bare_extraction(downloaded, no_fallback=True, favor_precision=True, include_formatting=False, max_tree_size=5)
wow

{'title': None,
 'author': None,
 'url': None,
 'hostname': None,
 'description': None,
 'sitename': None,
 'date': None,
 'categories': [],
 'tags': [],
 'fingerprint': None,
 'id': None,
 'license': None,
 'body': None,
 'comments': '',
 'commentsbody': None,
 'raw_text': None,
 'text': '{\n"_": "messages.channelMessages",\n"flags": 0,\n"inexact": false,\n"pts": 156835,\n"count": 62308,\n"messages": [\n{\n"_": "message",\n"flags": 18048,\n"out": false,\n"mentioned": false,\n"media_unread": false,\n"silent": false,\n"post": true,\n"from_scheduled": false,\n"legacy": false,\n"edit_hide": false,\n"pinned": false,\n"noforwards": false,\n"id": 63018,\n"peer_id": {\n"_": "peerChannel",\n"channel_id": 1036240821\n},\n"date": 1656105378,\n"message": "«Если с человеком работают три часа, значит, это нужно». В ФСБ заявили об отсутствии злоупотреблений при «фильтрации» граждан Украины\n\\n\n\\nДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, 

In [168]:
# diction = eval(wow)

In [228]:
answer = requests.get('https://tg.i-c-a.su/rss/meduzalive?limit=100').text
obj = BeautifulSoup(answer, features="xml")
obj

<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom">
<channel>
<title>Channel: meduzalive</title>
<link>https://tg.i-c-a.su</link>
<description>Telegram public channel RSS feed</description>
<pubDate>Sat, 25 Jun 2022 00:16:18 +0300</pubDate>
<lastBuildDate>Sat, 25 Jun 2022 00:16:18 +0300</lastBuildDate>
<atom:link href="https://tg.i-c-a.su/rss/meduzalive" rel="self" type="application/rss+xml"/>
<item>
<title>[Media] «Если с человеком работают три часа, значит, это нужно».</title>
<description>&lt;br/&gt;&lt;strong&gt;«Если с человеком работают три часа, значит, это нужно». В ФСБ заявили об отсутствии злоупотреблений при «фильтрации» граждан Украины&lt;/strong&gt;&lt;br /&gt;
&lt;br /&gt;
ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА, И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА&lt;br /&gt;
&lt;br /&gt;
Представит

In [223]:
print(obj.prettify())

<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom">
 <channel>
  <title>
   Channel: meduzalive
  </title>
  <link>
   https://tg.i-c-a.su
  </link>
  <description>
   Telegram public channel RSS feed
  </description>
  <pubDate>
   Sat, 25 Jun 2022 00:16:18 +0300
  </pubDate>
  <lastBuildDate>
   Sat, 25 Jun 2022 00:16:18 +0300
  </lastBuildDate>
  <atom:link href="https://tg.i-c-a.su/rss/meduzalive" rel="self" type="application/rss+xml"/>
  <item>
   <title>
    [Media] «Если с человеком работают три часа, значит, это нужно».
   </title>
   <description>
    &lt;br/&gt;&lt;strong&gt;«Если с человеком работают три часа, значит, это нужно». В ФСБ заявили об отсутствии злоупотреблений при «фильтрации» граждан Украины&lt;/strong&gt;&lt;br /&gt;
&lt;br /&gt;
ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА, И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ, ВЫПОЛ

In [230]:
obj.rss.channel.item.description.string

'<br/><strong>«Если с\xa0человеком работают три часа, значит, это нужно». В\xa0ФСБ заявили об\xa0отсутствии злоупотреблений при «фильтрации» граждан Украины</strong><br />\n<br />\nДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА, И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА<br />\n<br />\nПредставитель ФСБ Михаил Шелудяев назвал обоснованными длительные проверки, которым подвергаются украинские беженцы при пересечении границы с\xa0Россией. Шелудяев отвечал на\xa0слова правозащитницы Светланы Ганнушкиной о\xa0многочисленных жалобах украинцев на\xa0нарушения их\xa0прав в\xa0«фильтрационных» пунктах, в\xa0том числе на\xa0избиения, незаконные задержания, длительный контроль и\xa0раздевание во\xa0время проверок<br />\n<br />\n<a href="https://mdza.io/srSm3DFWrec" target="_blank" rel="nofollow">https://mdza.io/srSm3DFWrec</a><br/><br/><blockquote cite="https://mdza.i

In [231]:
text = obj.item.description

In [233]:
text = text.get_text()

In [234]:
from bs4 import UnicodeDammit

In [253]:
dammit = UnicodeDammit(text).unicode_markup.strip()

In [254]:
dammit

'<br/><strong>«Если с\xa0человеком работают три часа, значит, это нужно». В\xa0ФСБ заявили об\xa0отсутствии злоупотреблений при «фильтрации» граждан Украины</strong><br />\n<br />\nДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА, И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА<br />\n<br />\nПредставитель ФСБ Михаил Шелудяев назвал обоснованными длительные проверки, которым подвергаются украинские беженцы при пересечении границы с\xa0Россией. Шелудяев отвечал на\xa0слова правозащитницы Светланы Ганнушкиной о\xa0многочисленных жалобах украинцев на\xa0нарушения их\xa0прав в\xa0«фильтрационных» пунктах, в\xa0том числе на\xa0избиения, незаконные задержания, длительный контроль и\xa0раздевание во\xa0время проверок<br />\n<br />\n<a href="https://mdza.io/srSm3DFWrec" target="_blank" rel="nofollow">https://mdza.io/srSm3DFWrec</a><br/><br/><blockquote cite="https://mdza.i